<img src="IMG_4380.png" width="750" height="1000">

In [14]:
import numpy as np
import heapq

distance_matrix = [
    [np.inf, 20, 18, 12, 8],
    [5, np.inf, 14, 7, 11],
    [12, 18, np.inf, 6, np.inf],
    [11, 17, 9, np.inf, 14],
    [10, 13, np.inf, 8, np.inf]
]

def array_to_sets(arr):
    for i in range(len(arr)):
        arr[i] += 1
    return arr

def branch_and_bound_tsp(distance_matrix):
    n = len(distance_matrix) # Количество городов
    min_distance = float('inf') # Начальное минимальное расстояние
    best_route = None # Лучший путь
    def tsp(node, visited, cost, path):
        nonlocal min_distance, best_route # Используем переменные, которые находятся до объявления функции
        if len(path) == n:
            cost += distance_matrix[node][0] # Если прошли все города, то добавляем стоимость возврата в начальный город
            if cost < min_distance:
                min_distance = cost 
                best_route = path + [0]
            return
        for next_node in range(n): # Перебор всех городов
            if next_node not in visited:
                tsp(next_node, visited | {next_node}, cost + distance_matrix[node][next_node], path + [next_node]) # Запускаем итерацию для следущего города
                # Добавляем город в посещенные, добавляем стоимость перехода в следущий город, добавляем город в текущий маршрут
    tsp(0, {0}, 0, [0])
    print(array_to_sets(best_route), min_distance)
branch_and_bound_tsp(distance_matrix)

[1, 5, 4, 3, 2, 1] 48


In [33]:
import pandas as pd
import numpy as np

def array_to_sets_modif(arr):
    # Если это скалярное значение (число)
    if np.isscalar(arr):
        return arr
    
    # Если это список/массив
    if isinstance(arr, (list, np.ndarray)):
        if len(arr) > 0 and isinstance(arr[0], (list, tuple)):
            # Для двумерного массива
            return [[x + 1 for x in row] for row in arr]
        else:
            # Для одномерного массива
            return [x + 1 for x in arr]
    
    return arr

def in_deep_matrix(p, y, x): # Удаление строки и столбца
    return p.drop([y], axis=0).drop([x], axis=1)

def reduction_matrix(p): # Приведение матрицы
    bottom_line = 0
    
    for index, row in p.iterrows():
        min_val = row.min() # Минимальное значение в текущей строке
        if np.isinf(min_val): # если все элементы бесконечность, то возвращаем бесконечность
            return np.inf        
        bottom_line += min_val
        for key, value in row.items(): # Вычитаем минимальное значение из строки
             p[key][index] -= min_val

    for key, value in p.items(): # Цикл по всем столбцам матрицы
        min_val = value.min() # Минимальное значение в столбце
        if np.isinf(min_val): # если все элементы бесконечность, то возвращаем бесконечность
            return np.inf     
        bottom_line += min_val # Добавляем минимальное значение в столбце к нижней границе
        for index, row in value.items(): # Вычитаем минимальное значение
            p[key][index] -= min_val

    return bottom_line

def partition_matrix(p):
    max_sum = 0
    index_i = None
    index_j = None
    for index, row in p.iterrows():
        for key, value in p.items():
            if p[key][index] == 0: # Является ли нулевым
                min_i = np.inf
                min_j = np.inf

                for k, v in p[key].items():
                    if k != index and v < min_i: # Если не на диагонали и меньше минимума
                        min_i = v

                for k, v in p.loc[index].items():
                    if k != key and v < min_j: # Если не на диагонали и меньше минимума
                        min_j = v
                        
                l = min_i + min_j
                if l > max_sum: # Если сумма минимумов оказалась больше максимума, то обновляем его
                    max_sum = l
                    index_i = index
                    index_j = key
                
    return [index_i, index_j, max_sum]

def reverse_index(l, i, j): 
    '''
    Функция помогает понять к какому городу приведет цепочка из города i из какого города цепочка
    приходит в город j\n
    l - список выбранных ребер\n
    i - идекс города отправления для нового ребра\n
    j - индекс города назначения для нового ребра\n
    '''
    def in_dict(d, v): # Проходим по цепочке родитель наследник
        while v in d:
            v = d[v]
        return v
    
    ln = len(l)
    d1 = {l[k][0]: l[k][1] for k in range(0, ln, 1)} # Отображение из какого города в какой ведет ребро
                                                     # если l = [(1,2), (3,4)], то d1 = {1:2,3:4}
    d2 = {l[k][1]: l[k][0] for k in range(0, ln, 1)} # Отображение в обратном направлении

    # Первый элемент - конечный город из i
    # Второй элемент - начальный город из j
    return [in_dict(d1, i), in_dict(d2, j)] 

def evaluation_matrix(p, res, bottom_line):
    '''
    Рекурсивная реализация метода ветвей и границ\n
    p - матрица расстояний\n
    res - словарь для хранения результатов:
    \t globalmin - минимальная длинная найденного маршрута\n
    \t best_result -лучший найденный маршрут\n
    \t local_result -текущий частичный маршрут\n
    \t steps -счётчик шагов\n
    bottom_line - текущая нижняя граница стоимости\n
    '''
    if len(p) == 1: # Если матрица единичная, то это последний город в маршрутте
        res['steps'] += 1
        bottom_line += p.iat[0, 0] # Добавляем последний элемент к общей стоимости

        if bottom_line < res['global_min']: # Если текущая стоимость меньше найденной
            res['global_min'] = bottom_line # Обновляем минимальную стоимость
            res['local_result'].append([p.index[0], p.columns[0]]) # Добавляем последнее ребро в маршрут     
            res['best_result'] = res['local_result'].copy() # Сохраняем копию маршрута как лучший результат
            print('Лучшее решение найдено:', bottom_line, array_to_sets_modif(res['best_result']), 'шаг:', res['steps'])         
        return # Завершение рекурсии для данного ветвления
    
    bottom_line += reduction_matrix(p) # Выполняем приведение
    if np.isinf(bottom_line): # Если доступных путей больше нет, то завершаем ветвление
        return # Завершение рекурсии для данного ветвления
    
    max_sum = 0
    while True:
        res['steps'] += 1
        i, j, max_sum = partition_matrix(p) # Выбор ребра для ветвления
        if i is None: # Если нет нулевых элементов, то завершаем
            return # Завершение рекурсии для данного ветвления

        v_len = len(res['local_result']) # Запоминаем текущую длинну маршрута
        if bottom_line < res['global_min']: # Если текущая оценка меньше известного решения
            res['local_result'].append([i, j]) # Добавляем ребро в маршрут
            p1 = in_deep_matrix(p, i, j) # Уменьшеная матрица(удалили строку и столбец)
            if len(p1) > 2: # Если оставшееся число городов больше двух
                i_reverse, j_reverse = reverse_index(res['local_result'], i, j) # Находим ребро, которое может создать подцикл 
                                                                                # Так как мы всегда добавляем ребра только те, которые мы хотим, то 
                                                                                # остается только убрать ребра потенциально образующие цикл
                p1[j_reverse][i_reverse] = np.inf # Ставим значение бесконечность(запрещаем его)
            evaluation_matrix(p1, res, bottom_line) # Продолжаем строить маршрут

        if res['global_min'] < bottom_line + max_sum: # Если оценка текущего поддерева хуже лучшего решения - отсекаем
            return # Завершение рекурсии для данного ветвления
        p[j][i] = np.inf # Рассматриваем версию без ребра i,j
        res['local_result'] = res['local_result'][:v_len] # Возвращаемся в состояние до добавления ребра
        
def return_res(res):
    l = res['best_result']
    if l:
        d = {l[k][0]: l[k][1] for k in range(len(l))}
        li = []
        a = l[0][0]
        for v in range(len(l)):
            li.append(a)
            a = d[a]
        return li
    else:
        return []

# Заданная матрица расстояний
distance_matrix = [
    [np.inf, 20, 18, 12, 8],
    [5, np.inf, 14, 7, 11],
    [12, 18, np.inf, 6, np.inf],
    [11, 17, 9, np.inf, 14],
    [10, 13, np.inf, 8, np.inf]
]

# Создаем DataFrame из матрицы
f1 = pd.DataFrame(distance_matrix, index=range(5), columns=range(5))
print(f1)


# Словарь
res = {'global_min': np.inf, 'best_result': [], 'local_result': [], 'steps': 0}

# Запуск алгоритма
evaluation_matrix(f1, res, 0)

# Вывод результатов
print('\nФинальный результат:')
print('Минимальная длина пути:', res['global_min'])
print('Оптимальный маршрут:', array_to_sets(return_res(res)))
print('Всего шагов:', res['steps'])

      0     1     2     3     4
0   inf  20.0  18.0  12.0   8.0
1   5.0   inf  14.0   7.0  11.0
2  12.0  18.0   inf   6.0   inf
3  11.0  17.0   9.0   inf  14.0
4  10.0  13.0   inf   8.0   inf
Лучшее решение найдено: 48.0 [[4, 3], [1, 5], [2, 1], [3, 2], [5, 4]] шаг: 5

Финальный результат:
Минимальная длина пути: 48.0
Оптимальный маршрут: [4, 3, 2, 1, 5]
Всего шагов: 5


In [ ]:
print(9+8+5+18+8) # Проверили, что сумма рёбер действительно равняется 48

48
